Задание: https://paper.dropbox.com/doc/vQ0SBXmxWRUTX7E3p1spp

In [106]:
import numpy as np
import numpy.random as rnd
import numpy.linalg as la
import matplotlib.pyplot as plt
import matplotlib as mpl

import scipy.stats as scs
import statsmodels.api as sm
from scipy.optimize import minimize, NonlinearConstraint
from statsmodels.distributions.empirical_distribution import ECDF

import multiprocessing as mp
from tqdm import tqdm
from time import time

from IPython.core.interactiveshell import InteractiveShell

# InteractiveShell.ast_node_interactivity = "all"
# %matplotlib notebook
%matplotlib inline
mpl.rc('text', usetex=True)
mpl.rc('font', size=14)

# Задание 7

## 7.1

In [96]:
def f(x):
    if abs(x[0]) < np.finfo(x[0]).eps or abs(x[1]) < np.finfo(x[1]).eps:
        return 0
    return x[0]**3 * np.sin(1 / x[0]) + 10 * x[0] * x[1]**4 * np.cos(1 / x[1])

In [122]:
N_REP = 10**5

y_min = np.inf
x_min = np.array([np.nan, np.nan])
for i in tqdm(range(N_REP)):
    r = np.sqrt(rnd.rand())
    phi = 2 * np.pi * rnd.rand()
    x = np.array([r * np.cos(phi), r * np.sin(phi)])
    y = f(x)
    if y < y_min:
        y_min = y
        x_min = x

print('f_min = {}'.format(y_min))
print('(x1_min, x2_min) = ({}, {})'.format(x_min[0], x_min[1]))

100%|██████████| 100000/100000 [00:04<00:00, 22108.80it/s]

f_min = -1.28506745476355
(x1_min, x2_min) = (-0.3727682725469592, 0.927894112624122)


## 2.2

In [42]:
def ros_func(x):
    return (x[0] - 1)**2 + 100 * (x[1] - x[0]**2)**2

In [70]:
def sa_optim(func, x0, sigma=1, k=0.99, t0=1, t_min=1e-5, verbose_every=100):
    x = x0
    y = func(x0)
    temp = t0
    it = 0
    steps = 0
    
    while temp > t_min:
        x_new = x + sigma * rnd.randn(*x.shape)
        y_new = func(x_new)
        prob = min(1, np.exp((y - y_new) / temp))
        if rnd.rand() <= prob:
            x = x_new
            y = y_new
            steps += 1
        temp *= k
        it += 1
        if it % verbose_every == 0:
            print('\rt: {}'.format(temp), end='')
    print('\niterations: {}\t steps: {}'.format(it, steps))
    return {'x_min': x, 'f_min': y}

In [72]:
%%time

opt_res = sa_optim(ros_func,
    np.array([0, 0]),
    sigma = 0.05,
    k = 0.999,
    t0 = 1,
    t_min = 1e-7,
    verbose_every=100
)
print(opt_res['x_min'], opt_res['f_min'])

t: 1.010090852990375e-077
iterations: 16111	 steps: 1508
[0.99927707 0.99841208] 2.5557696440928405e-06
CPU times: user 639 ms, sys: 95 ms, total: 734 ms
Wall time: 636 ms


## 7.3

In [99]:
def grad_f(x):   
    g1 = 3 * x[0]**2 * np.sin(1 / x[0]) - x[0] * np.cos(1 / x[0]) + 10 * x[1]**4 * np.cos(1 / x[1])
    g2 = 40 * x[0] * x[1]**3 * np.cos(1 / x[1]) + 10 * x[0] * x[1]**2 * np.sin(1 / x[1])
    
    return np.array([g1, g2])

In [87]:
def grad_ros(x):
    g1 = 2 * (x[0] - 1) - 400 * x[0] * (x[1] - x[0]**2)
    g2 = 200 * (x[1] - x[0]**2)
    
    return np.array([g1, g2])

In [88]:
minimize(ros_func, np.array([0, 0]), jac=grad_ros)

      fun: 7.717288356613562e-13
 hess_inv: array([[0.49480256, 0.98953879],
       [0.98953879, 1.98387918]])
      jac: array([ 3.92841201e-06, -2.83120873e-06])
  message: 'Optimization terminated successfully.'
     nfev: 24
      nit: 19
     njev: 24
   status: 0
  success: True
        x: array([0.99999913, 0.99999825])

In [121]:
constr = NonlinearConstraint(fun=lambda x: (x**2).sum(),
                             lb = -np.inf,
                             ub = 1,
                             jac=lambda x: 2 * x.sum())
minimize(f, np.array([-0.36, -0.93]), jac=grad_f, constraints=constr)

     fun: -1.6060013046111983
     jac: array([5.5302794 , 8.72028325])
 message: 'Iteration limit exceeded'
    nfev: 1093
     nit: 101
    njev: 101
  status: 9
 success: False
       x: array([-0.28067219, -1.00938724])

# Задание 8